In [0]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier,VotingClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import *
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC,OneClassSVM

from sklearn.model_selection import train_test_split
from sklearn import metrics

from sklearn.datasets import load_boston

boston = load_boston()
import matplotlib.pyplot as plt

%matplotlib inline

## Loading data and pre-processing 


In [7]:
df_boston = pd.DataFrame(boston.data,columns=boston.feature_names)
df_boston['target'] = pd.Series(boston.target)
df = df_boston
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


| Feature | Description                                                           |
|---------|-----------------------------------------------------------------------|
| crim    | per capita crime rate by town.                                        |
| zn      | proportion of residential land zoned for lots over 25,000     |
|         |                                                                       |
| indus   | proportion of non-retail business acres per town.                     |
| chas    | Charles River dummy variable (= 1 if tract bounds river; 0otherwise|
|         |                                                                       |
| nox     | nitrogen oxides concentration (parts per 10 million).                 |
| lstat   | median value of owner-occupied homes in $$ 1000s.                       |
| rm      | average number of rooms per dwelling.                                 |
| age     | proportion of owner-occupied units built prior to 1940.               |
| dis     | weighted mean of distances to five Boston employment centres.         |
| rad     | index of accessibility to radial highways.                            |
| tax     | full-value property-tax rate per \$10,000.                             |
| ptratio | pupil-teacher ratio by town.                                          |
| black   | 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town.       |
| lstat   | lower   status of the population (percent).                           |
| medv    | median   value of owner-occupied homes in $1000s.                     |

###  surely no one cares about nox,chas 

In [36]:
df =df_boston.drop(['NOX','CHAS','INDUS','DIS','RAD'],axis=1)
df_byCRIM = df.sort_values('CRIM')

own = np.zeros(len(df_byCRIM))
own[:20] = 1 
df_byCRIM['own'] = own

df = df_byCRIM.sort_index()
df_byCRIM.head(30)

,CRIM,ZN,RM,AGE,TAX,PTRATIO,B,LSTAT,target,own
0,0.00632,18.0,6.575,65.2,296.0,15.3,396.90,4.98,24.0,1.0
284,0.00906,90.0,7.088,20.8,285.0,15.3,394.72,7.85,32.2,1.0
285,0.01096,55.0,6.453,31.9,300.0,15.3,394.72,8.23,22.0,1.0
341,0.01301,35.0,7.241,49.3,284.0,15.5,394.74,5.49,32.7,1.0
55,0.01311,90.0,7.249,21.9,226.0,17.9,395.93,4.81,35.4,1.0
54,0.01360,75.0,5.888,47.6,469.0,21.1,396.90,14.80,18.9,1.0
195,0.01381,80.0,7.875,32.0,255.0,14.4,394.23,2.97,50.0,1.0
57,0.01432,100.0,6.816,40.5,256.0,15.1,392.90,3.95,31.6,1.0
194,0.01439,60.0,6.604,18.8,265.0,15.6,376.70,4.38,29.1,1.0
348,0.01501,80.0,6.635,29.7,280.0,17.0,390.94,5.99,24.5,1.0


In [0]:
if len(df_byCRIM) > 10:
   df_client = df_byCRIM[:10]
   df_listings = df_byCRIM[10:]

In [10]:
df_new = pd.concat([df_client, df] ,axis=0)
df_new.head(14)

,CRIM,ZN,RM,AGE,TAX,PTRATIO,B,LSTAT,target,own
0,0.00632,18.0,6.575,65.2,296.0,15.3,396.90,4.98,24.0,1.0
284,0.00906,90.0,7.088,20.8,285.0,15.3,394.72,7.85,32.2,1.0
285,0.01096,55.0,6.453,31.9,300.0,15.3,394.72,8.23,22.0,1.0
341,0.01301,35.0,7.241,49.3,284.0,15.5,394.74,5.49,32.7,1.0
55,0.01311,90.0,7.249,21.9,226.0,17.9,395.93,4.81,35.4,1.0
54,0.01360,75.0,5.888,47.6,469.0,21.1,396.90,14.80,18.9,1.0
195,0.01381,80.0,7.875,32.0,255.0,14.4,394.23,2.97,50.0,1.0
57,0.01432,100.0,6.816,40.5,256.0,15.1,392.90,3.95,31.6,1.0
194,0.01439,60.0,6.604,18.8,265.0,15.6,376.70,4.38,29.1,1.0
348,0.01501,80.0,6.635,29.7,280.0,17.0,390.94,5.99,24.5,1.0


In [11]:
X = df_new.drop(['own'],axis=1) # Features
y = df_new['own'] # Target variable

def non_shuffling_train_test_split(X, y, test_size):
    i = int((1 - test_size) * X.shape[0]) + 1
    X_train, X_test = np.split(X, [i])
    y_train, y_test = np.split(y, [i])
    return X_train, X_test, y_train, y_test


X_train, X_test, y_train, y_test = non_shuffling_train_test_split(X, y, test_size=0.85)

X_train.head(3)

,CRIM,ZN,RM,AGE,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,6.575,65.2,296.0,15.3,396.90,4.98,24.0
284,0.00906,90.0,7.088,20.8,285.0,15.3,394.72,7.85,32.2
285,0.01096,55.0,6.453,31.9,300.0,15.3,394.72,8.23,22.0


## Testing models


In [12]:
clf = DecisionTreeClassifier()
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


Accuracy: 0.997716894977169


In [0]:
#y_pred

In [0]:
#np.array(y_test)

In [15]:
clf0 = DecisionTreeClassifier(criterion="entropy", max_depth=8)
clf0.fit(X_train,y_train)
y_pred = clf0.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


Accuracy: 0.997716894977169


In [42]:
clf = LinearSVC(random_state=0, tol=1e-5)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.3310502283105023


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [43]:
from sklearn.svm import SVC
clf = SVC(random_state=0, tol=1e-5)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9817351598173516


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [0]:
# y_pred

In [0]:
# np.array(y_test)

In [19]:
clf = OneClassSVM()
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.01141552511415525


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [20]:
for i in range(1,15):
  knn = KNeighborsClassifier(n_neighbors=i)
  knn.fit(X_train, y_train)
  y_pred = knn.predict(X_test)
  print('I: {0}, Accuracy: {1}'.format(i, metrics.accuracy_score(y_test, y_pred)))

I: 1, Accuracy: 0.6095890410958904
I: 2, Accuracy: 0.6484018264840182
I: 3, Accuracy: 0.636986301369863
I: 4, Accuracy: 0.9794520547945206
I: 5, Accuracy: 0.958904109589041
I: 6, Accuracy: 0.9634703196347032
I: 7, Accuracy: 0.958904109589041
I: 8, Accuracy: 0.958904109589041
I: 9, Accuracy: 0.9611872146118722
I: 10, Accuracy: 0.9680365296803652
I: 11, Accuracy: 0.958904109589041
I: 12, Accuracy: 0.9703196347031964
I: 13, Accuracy: 0.9680365296803652
I: 14, Accuracy: 0.9703196347031964


In [21]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print('I: {0}, Accuracy: {1}'.format(i, metrics.accuracy_score(y_test, y_pred)))


I: 14, Accuracy: 0.958904109589041


In [0]:
#y_pred

In [0]:
# np.array(y_test)

In [24]:
G_NB = GaussianNB()
G_NB.fit(X_train, y_train)
y_pred = G_NB.predict(X_test)
print('GaussianNB: Accuracy: {0}'.format(metrics.accuracy_score(y_test, y_pred)))

GaussianNB: Accuracy: 0.9748858447488584


In [25]:
B_NB = BernoulliNB()
B_NB.fit(X_train, y_train)
y_pred = B_NB.predict(X_test)
print('BernoulliNB: Accuracy: {0}'.format(metrics.accuracy_score(y_test, y_pred)))

BernoulliNB: Accuracy: 0.9680365296803652


In [44]:
M_NB = MultinomialNB()
M_NB.fit(X_train, y_train)
y_pred = M_NB.predict(X_test)
print('MultinomialNB: Accuracy: {0}'.format(metrics.accuracy_score(y_test, y_pred)))

MultinomialNB: Accuracy: 0.8721461187214612


In [45]:
C_NB = ComplementNB()
C_NB.fit(X_train, y_train)
y_pred = C_NB.predict(X_test)
print('ComplementNB: Accuracy: {0}'.format(metrics.accuracy_score(y_test, y_pred)))

ComplementNB: Accuracy: 0.8698630136986302


In [28]:
clf1 = LogisticRegression(solver='lbfgs', multi_class='multinomial',                        random_state=1)
clf2 = RandomForestClassifier(n_estimators=50, random_state=1)
clf3 = GaussianNB()

eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='hard')

for clf, label in zip([clf1, clf2, clf3, eclf], ['Logistic Regression', 'Random Forest', 'naive Bayes', 'Ensemble']):
     scores = cross_val_score(clf, X, y, cv=5, scoring='accuracy')
     print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Accuracy: 0.95 (+/- 0.03) [Logistic Regression]
Accuracy: 0.99 (+/- 0.00) [Random Forest]
Accuracy: 0.93 (+/- 0.04) [naive Bayes]


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Accuracy: 0.97 (+/- 0.03) [Ensemble]


In [29]:
clf1 =GradientBoostingClassifier()
clf2 = RandomForestClassifier(n_estimators=20, random_state=1)
clf3 = GaussianNB()

eclf = VotingClassifier(estimators=[('GradientBoostingClassifier', clf1), ('rf', clf2), ('gnb', clf3),('DecisionTreeClassifier', clf0),('knn', knn),('BernoulliNB',B_NB)], voting='hard') #soft
l1 = [clf1, clf2, clf3, B_NB, knn, B_NB,eclf]
l2 = ['GradientBoostingClassifier', 'Random Forest', 'naive Bayes','DecisionTreeClassifier','KNN','BernoulliNB', 'Ensemble']
for clf, label in zip(l1,l2):
     scores = cross_val_score(clf, X, y, cv=5, scoring='accuracy')
     print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

Accuracy: 1.00 (+/- 0.00) [GradientBoostingClassifier]
Accuracy: 1.00 (+/- 0.00) [Random Forest]
Accuracy: 0.93 (+/- 0.04) [naive Bayes]
Accuracy: 0.94 (+/- 0.00) [DecisionTreeClassifier]
Accuracy: 0.95 (+/- 0.02) [KNN]
Accuracy: 0.94 (+/- 0.00) [BernoulliNB]
Accuracy: 0.99 (+/- 0.00) [Ensemble]


In [30]:
clf1 =GradientBoostingClassifier() 
clf2 = RandomForestClassifier(n_estimators=20, random_state=1)
clf3 = GaussianNB()
clf4 = AdaBoostClassifier()

eclf = VotingClassifier(estimators=[('GradientBoostingClassifier', clf1), ('rf', clf2), ('gnb', clf3),('DecisionTreeClassifier', clf0),('knn', knn),('BernoulliNB',B_NB),('AdaBoostClassifier',clf4)], voting='soft') #soft
l1 = [clf1, clf2, clf3, B_NB, knn, B_NB,clf4,eclf]
l2 = ['GradientBoostingClassifier', 'Random Forest', 'naive Bayes','DecisionTreeClassifier','KNN','BernoulliNB','AdaBoostClassifier', 'Ensemble']
for clf, label in zip(l1,l2):
     scores = cross_val_score(clf, X, y, cv=5, scoring='accuracy')
     print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

Accuracy: 1.00 (+/- 0.00) [GradientBoostingClassifier]
Accuracy: 1.00 (+/- 0.00) [Random Forest]
Accuracy: 0.93 (+/- 0.04) [naive Bayes]
Accuracy: 0.94 (+/- 0.00) [DecisionTreeClassifier]
Accuracy: 0.95 (+/- 0.02) [KNN]
Accuracy: 0.94 (+/- 0.00) [BernoulliNB]
Accuracy: 1.00 (+/- 0.00) [AdaBoostClassifier]
Accuracy: 1.00 (+/- 0.00) [Ensemble]


In [31]:
eclf.fit(X_train,y_train)
#np.array(eclf.predict(X_test) == y_test)

VotingClassifier(estimators=[('GradientBoostingClassifier',
                              GradientBoostingClassifier(criterion='friedman_mse',
                                                         init=None,
                                                         learning_rate=0.1,
                                                         loss='deviance',
                                                         max_depth=3,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
       

## Final results 


In [32]:
#X_train, X_test, y_train, y_test = non_shuffling_train_test_split(X, y, test_size=0.85)

df_prediction = X_test.copy()
df_prediction['prob'] = eclf.predict_proba(X_test)[:,1]
df_prediction['own'] = y_test
df_prediction = df_prediction[df_prediction['own']==0]
df_prediction= df_prediction.sort_values(['prob'],ascending=False,axis=0)
df_prediction.head(5)

,CRIM,ZN,RM,AGE,TAX,PTRATIO,B,LSTAT,target,prob,own
193,0.02187,60.0,6.800,9.9,265.0,15.6,393.37,5.03,31.1,0.839557,0.0
203,0.03510,95.0,7.853,33.2,224.0,14.7,392.78,3.81,48.5,0.403947,0.0
198,0.03768,80.0,7.274,38.3,329.0,12.6,392.20,6.62,34.6,0.336496,0.0
199,0.03150,95.0,6.975,15.3,402.0,17.0,396.90,4.56,34.9,0.303947,0.0
344,0.03049,55.0,6.874,28.1,370.0,17.6,387.97,4.61,31.2,0.295907,0.0


In [33]:
print('next five places to consider buying are listing #{0}'.format(str(list(df_prediction.index[:5]))))

next five places to consider buying are listing #[193, 203, 198, 199, 344]


In [70]:
x = df.describe()
x.iloc[[1,2]]

,CRIM,ZN,RM,AGE,TAX,PTRATIO,B,LSTAT,target,own
mean,3.613524,11.363636,6.284634,68.574901,408.237154,18.455534,356.674032,12.653063,22.532806,0.039526
std,8.601545,23.322453,0.702617,28.148861,168.537116,2.164946,91.294864,7.141062,9.197104,0.195035


# Let's try for fancy new age methods 

### if you can read this, this is not done yet.... gotta learn new stuff. switching to movielens dataset, since i can copy,paste and modify others code to learn quickly

In [87]:
import gc 
gc.collect()
df_boston = pd.DataFrame(boston.data,columns=boston.feature_names)
df_boston['target'] = pd.Series(boston.target)
df = df_boston

df =df_boston.drop(['NOX','CHAS','INDUS','DIS','RAD'],axis=1) 
# accuracy                           0.97       438
# df =df_boston.drop(['NOX','CHAS'],axis=1)   
# accuracy                           0.97       438

df_byCRIM = df.sort_values('CRIM')

own = np.zeros(len(df_byCRIM))
own[:20] = 1 
df_byCRIM['own'] = own

df = df_byCRIM.sort_index()
df.head(15)

,CRIM,ZN,RM,AGE,TAX,PTRATIO,B,LSTAT,target,own
0,0.00632,18.0,6.575,65.2,296.0,15.3,396.90,4.98,24.0,1.0
1,0.02731,0.0,6.421,78.9,242.0,17.8,396.90,9.14,21.6,0.0
2,0.02729,0.0,7.185,61.1,242.0,17.8,392.83,4.03,34.7,0.0
3,0.03237,0.0,6.998,45.8,222.0,18.7,394.63,2.94,33.4,0.0
4,0.06905,0.0,7.147,54.2,222.0,18.7,396.90,5.33,36.2,0.0
5,0.02985,0.0,6.430,58.7,222.0,18.7,394.12,5.21,28.7,0.0
6,0.08829,12.5,6.012,66.6,311.0,15.2,395.60,12.43,22.9,0.0
7,0.14455,12.5,6.172,96.1,311.0,15.2,396.90,19.15,27.1,0.0
8,0.21124,12.5,5.631,100.0,311.0,15.2,386.63,29.93,16.5,0.0
9,0.17004,12.5,6.004,85.9,311.0,15.2,386.71,17.10,18.9,0.0


In [88]:
if len(df_byCRIM) > 10:
   df_client = df_byCRIM[:10]
   df_listings = df_byCRIM[10:]
   

df_new = pd.concat([df_client, df] ,axis=0)
X = df_new.drop(['own'],axis=1) # Features
y = df_new['own'] # Target variable

def non_shuffling_train_test_split(X, y, test_size):
    i = int((1 - test_size) * X.shape[0]) + 1
    X_train, X_test = np.split(X, [i])
    y_train, y_test = np.split(y, [i])
    return X_train, X_test, y_train, y_test


X_train, X_test, y_train, y_test = non_shuffling_train_test_split(X, y, test_size=0.85)

X_train.head(3)

,CRIM,ZN,RM,AGE,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,6.575,65.2,296.0,15.3,396.90,4.98,24.0
284,0.00906,90.0,7.088,20.8,285.0,15.3,394.72,7.85,32.2
285,0.01096,55.0,6.453,31.9,300.0,15.3,394.72,8.23,22.0


In [0]:
from sklearn.neural_network import BernoulliRBM
from sklearn.pipeline import Pipeline
from sklearn import linear_model, datasets, metrics

logistic = linear_model.LogisticRegression(solver='newton-cg', tol=1,
                                           multi_class='multinomial')

rbm = BernoulliRBM(random_state=0, verbose=True)
rbm_features_classifier = Pipeline(
    steps=[('rbm', rbm), ('logistic', logistic)])

In [90]:
rbm_features_classifier.fit(X_train, y_train)

[BernoulliRBM] Iteration 1, pseudo-likelihood = -2239.65, time = 0.01s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -5028.07, time = 0.01s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -7809.52, time = 0.00s
[BernoulliRBM] Iteration 4, pseudo-likelihood = -10589.56, time = 0.00s
[BernoulliRBM] Iteration 5, pseudo-likelihood = -13369.60, time = 0.00s
[BernoulliRBM] Iteration 6, pseudo-likelihood = -16147.02, time = 0.00s
[BernoulliRBM] Iteration 7, pseudo-likelihood = -18931.10, time = 0.00s
[BernoulliRBM] Iteration 8, pseudo-likelihood = -21712.89, time = 0.00s
[BernoulliRBM] Iteration 9, pseudo-likelihood = -24492.97, time = 0.00s
[BernoulliRBM] Iteration 10, pseudo-likelihood = -27281.41, time = 0.01s


Pipeline(memory=None,
         steps=[('rbm',
                 BernoulliRBM(batch_size=10, learning_rate=0.1,
                              n_components=256, n_iter=10, random_state=0,
                              verbose=True)),
                ('logistic',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='multinomial', n_jobs=None,
                                    penalty='l2', random_state=None,
                                    solver='newton-cg', tol=1, verbose=0,
                                    warm_start=False))],
         verbose=False)

In [91]:
y_pred = rbm_features_classifier.predict(X_test)
print("Logistic regression using RBM features:\n%s\n" % (
    metrics.classification_report(y_test, y_pred)))

Logistic regression using RBM features:
              precision    recall  f1-score   support

         0.0       0.97      1.00      0.98       424
         1.0       0.00      0.00      0.00        14

    accuracy                           0.97       438
   macro avg       0.48      0.50      0.49       438
weighted avg       0.94      0.97      0.95       438




/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
